In [0]:
!wget "https://sds-platform-private.s3-us-east-2.amazonaws.com/uploads/P23-Code-for-Mac-and-Linux.zip"

In [0]:
!unzip "P23-Code-for-Mac-and-Linux.zip"

In [1]:
cd Code\ for\ Mac\ and\ Linux

/content/Code for Mac and Linux


In [2]:
pwd

'/content/Code for Mac and Linux'

In [0]:
import torch
from torch.autograd import Variable
import cv2
from data import BaseTransform,VOC_CLASSES as labelmap
from ssd import build_ssd
import imageio

In [0]:
#Function for detecting Objects in a single frame
def detect(frame,net,transform):
  height,width = frame.shape[:2]
  frame_t = transform(frame)[0]
  x = torch.from_numpy(frame_t).permute(2,0,1)
  x = Variable(x.unsqueeze(0))
  
  y = net(x)
  detections = y.data
  scale = torch.Tensor([height,width,height,width])

  #detections = [batch,no. of classes,number of occurance,(score,x0,y0,x1,y1)]
  for i in range(detections.size(1)):
    j = 0
    while detections[0,i,j,0] >= 0.6:
      pt = (detections[0,i,j,1:]*scale).numpy()
      cv2.rectangle(frame,(int(pt[0]),int(pt[1])),(int(pt[2]),int(pt[3])),(0,255,0),2)
      cv2.putText(frame,labelmap[i-1],(int(pt[0]),int(pt[1])),cv2.FONT_HERSHEY_COMPLEX,2,(0,255,0),2,cv2.LINE_AA)
      j+=1
  return frame

In [8]:
#Creating the SSD Model
net = build_ssd('test')
net.load_state_dict(torch.load("ssd300_mAP_77.43_v2.pth",map_location = lambda storage, loc: storage))

#Creating the Transfromation Object
transform = BaseTransform(net.size,(104/256.0, 117/256.0, 123/256.0))

/content/Code for Mac and Linux/ssd.py:33: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  self.priors = Variable(self.priorbox.forward(), volatile=True)
/content/Code for Mac and Linux/layers/modules/l2norm.py:17: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  init.constant(self.weight,self.gamma)


In [0]:
#Dealing with the video
reader = imageio.get_reader("funny_dog.mp4")
fps = reader.get_meta_data()['fps']
writer = imageio.get_writer("converted.mp4",fps=fps)
for i,frame in enumerate(reader):
  frame = detect(frame,net.eval(),transform)
  writer.append_data(frame)
  print(i)

writer.close()